In [1]:
import pandas as pd

df = pd.read_csv('data_fixed_3.csv')

C:\Temp\ipykernel_14748\494984042.py:3: DtypeWarning: Columns (15,20,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_fixed_3.csv')


In [3]:
print(df['action_taken'].unique())
action_taken_mapping = {
    1: "Loan originated",
    2: "Application approved but not accepted",
    3: "Application denied",
    7: "Preapproval request denied",
    8: "Preapproval request approved but not accepted"
}

df['action_taken'] = df['action_taken'].map(action_taken_mapping)

[1 3 8 7 2]


In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = encoder.fit_transform(df[col].astype(str))

In [5]:
X = df.drop(columns=['action_taken'])  # Dropping non-relevant columns
Y = df['action_taken']  # Target variable

In [6]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(X , Y , test_size=0.2 , random_state=69 , shuffle=True)

In [7]:
print("X_Test Shape = " , x_test.shape)
print("Y_Test Shape = " , y_test.shape)
print("X_Train Shape = " , x_train.shape)
print("Y_Train Shape = " , y_train.shape)

X_Test Shape =  (1644176, 37)
Y_Test Shape =  (1644176,)
X_Train Shape =  (6576701, 37)
Y_Train Shape =  (6576701,)


In [8]:
%pip install catboost lightgbm
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Initialize classifiers with moderate complexity
cls1 = GradientBoostingClassifier(n_estimators=10)
cls2 = LGBMClassifier(n_estimators=10)
cls3 = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=10)
cls4 = AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=10)
cls5 = XGBClassifier(n_estimators=10, max_depth=6, eval_metric='logloss')
cls6 = RandomForestClassifier(n_estimators=10)
cls7 = ExtraTreesClassifier(n_estimators=10)
cls8 = DecisionTreeClassifier()
cls9 = CatBoostClassifier(logging_level='Silent', n_estimators=10)

# Fit the classifiers
cls1.fit(x_train, y_train)
cls2.fit(x_train, y_train)
cls3.fit(x_train, y_train)
cls4.fit(x_train, y_train)
cls5.fit(x_train, y_train)
cls6.fit(x_train, y_train)
cls7.fit(x_train, y_train)
cls8.fit(x_train, y_train)
cls9.fit(x_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.374752 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2860
[LightGBM] [Info] Number of data points in the train set: 6576701, number of used features: 37
[LightGBM] [Info] Start training from score -3.283618
[LightGBM] [Info] Start training from score -1.418379
[LightGBM] [Info] Start training from score -0.368248
[LightGBM] [Info] Start training from score -3.870345
[LightGBM] [Info] Start training from score -4.879006
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
# Predict results for each classifier
pred1 = cls1.predict(x_test)
pred2 = cls2.predict(x_test)
pred3 = cls3.predict(x_test)
pred4 = cls4.predict(x_test)
pred5 = cls5.predict(x_test)
pred6 = cls6.predict(x_test)
pred7 = cls7.predict(x_test)
pred8 = cls8.predict(x_test)
pred9 = cls9.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt  # Importing matplotlib.pyplot as plt

# Calculate accuracy for each classifier
acc_values = [
    accuracy_score(y_test, pred1),
    accuracy_score(y_test, pred2),
    accuracy_score(y_test, pred3),
    accuracy_score(y_test, pred4),
    accuracy_score(y_test, pred5),
    accuracy_score(y_test, pred6),
    accuracy_score(y_test, pred7),
    accuracy_score(y_test, pred8),
    accuracy_score(y_test, pred9)
]

classifier_names = [
    'GradientBoostingClassifier',
    'LGBMClassifier',
    'BaggingClassifier',
    'AdaBoostClassifier',
    'XGBClassifier',
    'RandomForestClassifier',
    'ExtraTreesClassifier',
    'DecisionTreeClassifier',
    'CatBoostClassifier'
]

# Sort accuracy values in descending order (high to low)
acc_sorted_indices = np.argsort(acc_values)[::-1]
acc_values_sorted = np.array(acc_values)[acc_sorted_indices]
classifier_names_acc_sorted = np.array(classifier_names)[acc_sorted_indices]

# Create a figure with a single subplot
plt.figure(figsize=(20, 6))

# Plot accuracy values (high to low)
plt.barh(classifier_names_acc_sorted, acc_values_sorted, color='skyblue')
plt.title('Accuracy of Classifiers (High to Low)')
plt.xlabel('Accuracy')
plt.xlim(0, 1)  # Assuming accuracy ranges from 0 to 1
plt.gca().invert_yaxis()  # Best performance on top

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Extract feature importances
bagging_importances = cls3.estimator_.feature_importances_
random_forest_importances = cls6.feature_importances_
lgbm_importances = cls2.feature_importances_

# Create subplots for the three classifiers
fig, axes = plt.subplots(1, 3, figsize=(20, 6), sharey=True)

# BaggingClassifier feature importance
axes[0].barh(X.columns, bagging_importances, color='lightcoral')
axes[0].set_title('BaggingClassifier Feature Importance')
axes[0].set_xlabel('Importance')
axes[0].invert_yaxis()

# RandomForestClassifier feature importance
axes[1].barh(X.columns, random_forest_importances, color='lightblue')
axes[1].set_title('RandomForestClassifier Feature Importance')
axes[1].set_xlabel('Importance')

# LGBMClassifier feature importance
axes[2].barh(X.columns, lgbm_importances, color='lightgreen')
axes[2].set_title('LGBMClassifier Feature Importance')
axes[2].set_xlabel('Importance')

# Adjust layout and display the plots
plt.tight_layout()
plt.show()